# PS1: GPU Population Dynamics Simulation (10 Points)

## Problem Statement
We're going to roll it back and do it all over again, this time on the GPU! As always, we have provided some starter code for you, and your job is to fill in the missing code as specified by the `#TODO#` blocks in the code. You can either just work in the ipynb OR you can work locally with the various files in this folder.

## Simulation Description (mostly repeated)
The world is split into `NUM_REGIONS` different regions. Each region is filled with `COMMUNITIES_PER_NUM_SPECIES` different communities of each of the `NUM_SPECIES` different species. Each of the communities is intialized with this information and a `population` and a `growth_rate` which is all packed into a `Species_Community` struct (note that the struct is a tad different to support GPU computations). Note that the struct also contains additional variables which you may or may not need to use depending upon your implementation.

```
typedef struct {
    int population;        // the population of a speciies
    int food_collected;    // the food collected in the current time period
    int region_of_world;   // region of this species community
    int species_type;      // type of species for this species community
    float growth_rate;     // growth_rate for this species community
    bool flag;             // flag in case helpful to have one (you may not need this)
    int helperi;           // flag in case helpful to have one (you may not need this)
    float helperf;         // flag in case helpful to have one (you may not need this)
} Species_Community;
```

For `NUM_TIMEPERIODS` the simulation runs. At each time period all of the members of each species calls the `food_oracle` inorder for everyone to `gather_all_food`. After all food is collected we can `update_all_populations` based on the amount of food gathered. In order to do so we need to first `compute_local_population_share` which is the percentage of all species WITHIN A SINGLE REGION that belong to a given species. We can then use that to `update_community_population` for each community of each species based on 3 rules as listed in later sections of this document.

When the simulation is done it prints out the populations of the various communities of species.

## Functions You'll Need To Implement
All functions you need to implement are in `util.h` and that is the only file you need to submit to gradescope!

First we'll implement helper functions that should be basically identical to your implementation for the CPU parallelism (with minor adjustments depending on your implemenation details of your other functions):

`update_community_population` (0.5 points)

For a given community, update its population based on the input `local_population_share` and the following three rules:
1. The change in population for a community is proportional to its growth_rate and local_population_share
2. If it has collected enough food to feed the population it grows, else it shrinks
3. If the population drops below 5 it goes extinct

`compute_local_population_share` (0.5 points)

Returns the population share for a given community. Population share is defined as the percentage of population in a region that is a given species across all communities of all species.

Then we'll implement the overall population update step: 

`update_all_populations` (2.5 points)

Updates the population for all communities of all species. Some quick hints/notes:
1. You will need to use compute_local_population_share and update_community_population
2. Make sure your logic is thread safe! Remember when you launch a kernel every line of the code is run by every thread in parallel!
3. Feel free to use helper functions if that makes your life easier!

Next we'll implement the food gathering step:

`gather_all_food` (2.5 points)

Each simualtion step we reset the food counts to 0 and then each member of the population tries to collect food using the food_oracle(). **On the GPU you are free to use whatever threading pattern you want!**

Then we'll implement the main kernel

`population_dynamics_kernel` (2 points)

This is a bit more complicated than the CPU but the premise is the same -- you want to do all NUM_TIME_PERIODS of gather_all_food and update_all_populations. However, as we are on the GPU, you'll want to use shared memory to speed things up, but then make sure to save things back to RAM once you're done!

Finally, we'll launch the main kernel from the main function:

`population_dynamics` (2 points)

Remember that we need to be careful about GPU vs. CPU memory! So set up GPU memory, run the kernel, and clean up GPU memory!

## Submission
Once you are done, download and submit (or just submit if you are working locally) your `util.h` file to **Courseworks**! As we do not have an autograder we can't use Gradescope.

## Notes and Hints
+ **DO NOT CHANGE FUNCTION DEFINITIONS** or you will break our grading scripts
+ See the syllabus for our course collaboration policy (long story short you are welcome to collaborate at a high level but please do not copy each others code).
+ If you are working in Colab, you can change the formatting of the code to different color schemes: just change the `%%cpp -n <filename>.h -s xcode` to a different `-s` flag. The list can be [found at this link](https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b).
+ Please reach out on Slack with any and all questions!

In [ ]:
# make sure CUDA is installed
!nvcc --version

In [ ]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

In [ ]:
# formatting helpers
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cuda -n helpers.h

#include <iostream>
#include <thread>
#include <vector>
#include <mutex>
#include <atomic>
#include <cstdlib>
#include <cmath>

// Some helpful constants
#define NUM_REGIONS 2
#define NUM_SPECIES 2
#define COMMUNITIES_PER_NUM_SPECIES 2
#define COMMUNITIES_PER_REGION (NUM_SPECIES*COMMUNITIES_PER_NUM_SPECIES)
#define TOTAL_COMMUNITIES (NUM_REGIONS*COMMUNITIES_PER_REGION)
#define MAX_STARTING_POPULATION 10
#define NUM_TIME_PERIODS 5

/************* 
 * With Presets of 2,2,2,10,5 you should get
 * 
 * ID[0]: of type [1]: in region [0]: had final population [11]
 * ID[1]: of type [1]: in region [0]: had final population [14]
 * ID[2]: of type [1]: in region [1]: had final population [237]
 * ID[3]: of type [0]: in region [1]: had final population [9]
 * ID[4]: of type [0]: in region [0]: had final population [97]
 * ID[5]: of type [0]: in region [0]: had final population [24]
 * ID[6]: of type [0]: in region [0]: had final population [5]
 * ID[7]: of type [1]: in region [1]: had final population [218]
 * 
 * OR, running on Mac you may get:
 * ID[0]: of type [1]: in region [0]: had final population [14]
 * ID[1]: of type [0]: in region [0]: had final population [81]
 * ID[2]: of type [0]: in region [0]: had final population [43]
 * ID[3]: of type [0]: in region [0]: had final population [23]
 * ID[4]: of type [0]: in region [1]: had final population [14]
 * ID[5]: of type [0]: in region [1]: had final population [170]
 * ID[6]: of type [1]: in region [1]: had final population [8]
 * ID[7]: of type [0]: in region [0]: had final population [5]
 * ************/

// data structure to store information about each species
// unlike in the CPU case we can't explicitly put atomics here
// or mutexs etc.
typedef struct {
    int population;        // the population of a speciies
    int food_collected;    // the food collected in the current time period
    int region_of_world;   // region of this species community
    int species_type;      // type of species for this species community
    float growth_rate;     // growth_rate for this species community
    bool flag;             // flag in case helpful to have one (you may not need this)
    int helperi;           // flag in case helpful to have one (you may not need this)
    float helperf;         // flag in case helpful to have one (you may not need this)
} Species_Community;

// food oracle function call
// call this with a community id to get a "random" amount of food back
// this represents one community member going out to get food
// we hardcode to 1 for determinism in testing but in theory should be random
__host__ __device__
int food_oracle(int community_id){return 1;};

// random range integer in range [0,max_range)
__host__
int rand_range(int max_range){return rand() % max_range;}
// random float in range [0,1]
__host__
float rand01(){return (float)rand() / (float)RAND_MAX;}

In [ ]:
%%cuda -n util.h

#include "helpers.h"
// I suggest testing with these set to 1 first!
#define NUM_BLOCKS 1
#define NUM_THREADS TOTAL_COMMUNITIES


// function to simulate population change for one community of one species
//
// Note: 1) The change in population for a community is proportional to 
//          its growth_rate and local_population_share
//       2) If it has collected enough food to feed the population it grows, else it shrinks
//       3) If the population drops below 5 it goes extinct
//
// Hint: this should remain basically unchanged from the CPU version (depending on the rest of your implementation)
__device__
void update_community_population(Species_Community *s_communities, int community_id, float local_population_share) {
  //
  // # TODO
  //
}

// function to find the local population share for one community of one species
//
// Note: 1) Population share is defined as the percentage of population in a region
//          that is a given species across all communities of all species
//
// Hint: this should remain basically unchanged from the CPU version (depending on the rest of your implementation)
__device__
float compute_local_population_share(Species_Community *s_communities, int community_id){
  //
  // # TODO
  //
  return 0;
}

// Updates the population for all communities of all species
//
// Note: 1) You will need to use compute_local_population_share and update_community_population
//       3) Make sure your logic is thread safe! Again this is likely to have a race condition!
//       4) Feel free to use helper functions if that makes your life easier!
__device__
void update_all_populations(Species_Community *communities){
  //
  // # TODO
  //
}

// function to simulate food gathering
// 
// Note: 1) Each round food starts at 0 and each member of the population tries to collect food
//       2) Please use food_oracle() to get a new amount of food for each member of the population
//       3) You can allocate threads to communites however you want!
//       3) All other implementation details are up to you! (Don't worry if your design isn't perfect!)
__device__
void gather_all_food(Species_Community *s_communities) {
  //
  // # TODO
  //
}

// the main kernel that computes the population dynamics over time
//
// Hints: 1) using shared memory may speed things up
//           but then make sure to save things back to RAM
//        2) make sure you do all NUM_TIME_PERIODS of gather_all_food
//           and update_all_populations
__global__
void population_dynamics_kernel(Species_Community *d_communities){
  //
  // # TODO
  //
}

// the main function
//
// Hints: set up GPU memory, run the kernel, clean up GPU memory
__host__
void population_dynamics(Species_Community *h_communities){
  //
  // #TODO
  //
}

In [ ]:
%%cuda -n run.cu

#include "util.h"

__host__
int main() {
  // initialize random and data
  srand(1337);
  Species_Community h_communities[TOTAL_COMMUNITIES];
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    h_communities[community_id].population = rand_range(MAX_STARTING_POPULATION) + 5;
    h_communities[community_id].region_of_world = rand_range(NUM_REGIONS);
    h_communities[community_id].species_type = rand_range(NUM_SPECIES);
    h_communities[community_id].growth_rate = rand01();
  }

  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type <<
                 "]: in region [" << h_communities[community_id].region_of_world << "]: had initial population [" << 
                 h_communities[community_id].population << "]" << std::endl;
  }
  
  // the main function
  population_dynamics(h_communities);

  // print the final populations
  std::cout << "\n---------\n---------\n";
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << h_communities[community_id].species_type <<
                 "]: in region [" << h_communities[community_id].region_of_world << "]: had final population [" << 
                 h_communities[community_id].population << "]" << std::endl;
  }
  return 0;
}

In [ ]:
!nvcc /content/src/run.cu -o run.exe
!./run.exe